# Exercise 4
## Creating a simple model

In this exercise we will create a simple logistic regression model from the scikit learn package.
We will then create some model evaluation metrics and test the predictions against those model evaluation metrics.
Let's load the feature data from the first excerice.

We should always approach training any machine learning model training as an iterative approach, beginning first with a simple model, and using model evaluation metrics to evaluate the performance of the models.

In [1]:
import pandas as pd
feats = pd.read_csv('data/bank_data_feats_e3.csv', index_col=0)
target = pd.read_csv('data/bank_data_target_e2.csv', index_col=0)

We first begin by creating a test and train dataset. We will train the data using the training dataset and evaluate the performance of the model on the test dataset. Later in the lesson we will add validation datasets that will help us tune the hyperparameters.

We will use a test_size = 0.2 which means that 20% of the data will be reserved for testing

In [2]:
from sklearn.model_selection import train_test_split
test_size = 0.2
random_state = 42
X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=test_size, random_state=random_state)

Let's make sure our dimensions are correct

In [3]:
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (3616, 32)
Shape of y_train: (3616, 1)
Shape of X_test: (905, 32)
Shape of y_test: (905, 1)


We fit our model first by instantiating it, then by fitting the model to the training data

In [4]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train['y'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

To test the model performance we will predict the outcome on the test features (X_test), and compare those outcomes to real values (y_test)

In [5]:
y_pred = model.predict(X_test)

Now let's compare against the true values. Let's start by using accuracy, accuracy is defined as the propotion of correct predictions out of the total predictions.

In [6]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'Accuracy of the model is {accuracy*100:.4f}%')

Accuracy of the model is 89.8343%


89.9448% - that's not bad with for a simple model with little feature engineering!

### Other evaluation metrics

Other common metrics in classification models are precision, recall, and f1-score.
Recall is defined as the proportion of correct positive predictions relative to total true postive values. Precision is defined as the proportion of correct positive predictions relative to total predicted postive values. F1 score is a combination of precision and recall, defined as 2 times the product of precision and recall, divided by the sum of the two.

It's useful to use these other evaluation metrics other than accuracy when the distribution of true and false values. We want these values to be as close to 1.0 as possible.

In [7]:
precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_pred=y_pred, y_true=y_test, average='binary')
print(f'Precision: {precision:.4f}\nRecall: {recall:.4f}\nfscore: {fscore:.4f}')

Precision: 0.5714
Recall: 0.2449
fscore: 0.3429


We can see here that while the accuracy is high, the recall is low, which means that we're missing most of the true positive values.

### Feature importances
   

We can look at which features are important by looking at the magnitude of the coefficients. Those with a larger coefficients will have a greater contribution to the result. Those with a positive value will make the result head toward the true result, that the customer will not subscribe. Features with a negative value for the coefficient will make the result heads towards a false result, that the customer will not subscribe to the product.

As a note, since the features were not normalized (having the same scale), the values for these coefficients shouls serve as a rough guide as to observe which features add predictive power.

In [8]:
coef_list = [f'{feature}: {coef}' for coef, feature in sorted(zip(model.coef_[0], X_train.columns.values.tolist()))]
for item in coef_list:
    print(item)

is_housing: -0.7751636157453201
is_loan: -0.7043690423505566
poutcome_failure: -0.6871730932787105
marital_married: -0.624121187394902
job_blue-collar: -0.583109720728534
marital_single: -0.5651918568886014
job_entrepreneur: -0.4042406812642158
education_primary: -0.3829909875697134
job_services: -0.32006834810680096
job_technician: -0.31884737843265887
education_secondary: -0.3155823691779084
job_unemployed: -0.28264585745779763
education_tertiary: -0.18896709097441358
job_self-employed: -0.18310115504031232
campaign: -0.0834630561369752
job_housemaid: -0.07855358815775614
job_management: -0.06220448577308672
month: -0.043352143714001354
age: -0.01689415430087044
job_admin.: -0.007887754728424817
day: -0.005291552451787906
balance: 4.878771152996165e-06
duration: 0.003946266910403602
previous: 0.008478116794752475
poutcome_other: 0.035290027327600296
job_student: 0.049705506837062666
is_default: 0.10729883531547348
contact_telephone: 0.3609828257670353
contact_cellular: 0.647384492243

We can see from the coefficients that a previous outcome is highly predictive of future outcome, with previous success being highly indicitive of customer subscription. Other important features are whether the custoer was reached by phone, or the type of job the customer has.